## Trying Transformers' BERT Model for sentiment analysis

#### Import Dependencies

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re

#### Instantiate Model

In [3]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

Downloading:   0%|          | 0.00/872k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

In [4]:
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

#### Encode and calculate sentiments

In [18]:
tokens = tokenizer.encode("I am okay this cold freezing weather.", return_tensors='pt')

In [19]:
tokens

tensor([[  101,   151, 10345, 44810, 10158, 10372, 19443, 12487, 21593, 24387,
           119,   102]])

In [20]:
result = model(tokens)
int(torch.argmax(result.logits))+1

3

#### Collect Reviews

In [27]:
r = requests.get("https://www.yelp.com/biz/mejico-sydney-2")
soup = BeautifulSoup(r.text, 'html.parser')
regex = re.compile('comment')
results = soup.find_all('p', {'class':regex})
reviews = [result.text for result in results]

In [35]:
int(torch.argmax(model(tokenizer.encode(reviews[0], return_tensors='pt')).logits))

3

In [41]:
def compute_sentiment(text):
    token = tokenizer.encode(text, return_tensors='pt')
    result = int(torch.argmax(model(token).logits))+1
    return result

In [42]:
output = []
for review in reviews:
    res = compute_sentiment(review)
    output.append((review, res))

#### Load reviews in dataframe

In [36]:
import pandas as pd
import numpy as np

In [43]:
df = pd.DataFrame(output, columns=["review", "sentiment"])

In [44]:
df

,review,sentiment
0,The food is fresh and tasty. The scallop cevi...,4
1,Don't come here expecting legit Mexican food b...,3
2,Out of all the restaurants that I tried in Syd...,5
3,We came here on a Thursday night @ 5pm and by ...,4
4,I was pleasantly surprised at what a great job...,5
5,Really nice (upmarket) Mexican restaurant. Goo...,4
6,Ordered feed me for $59 along with that.. Food...,2
7,"The best tasting cauliflower taco, mushroom ce...",5
8,Have been here twice and have absolutely loved...,5
9,If you're looking for a quiet little romantic ...,3
